<a href="https://colab.research.google.com/github/Atharvax16/Code-Qwen-AI-Studio/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import Markdown, display, update_display
import gradio as gr
from google.colab import userdata
import subprocess
from transformers import AutoModelForCausalLM, AutoTokenizer,TextIteratorStreamer


In [38]:
load_dotenv(override=True)
llama = os.getenv('LLAMA')
hf_token = userdata.get('HF_TOKEN')

if hf_token:
    print("Hugging Face token loaded successfully.")
else:
    print("Token not found in secrets.")

Hugging Face token loaded successfully.


In [4]:
openai = OpenAI(base_url = llama,api_key = 'ollama')

In [5]:
device = "cuda"

In [6]:
print("Loading CodeQwen model")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/CodeQwen1.5-7B-Chat",
    torch_dtype="auto",
    device_map="auto"
)

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/CodeQwen1.5-7B-Chat")

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

In [39]:
def llama_chat(message, history):
  stream = openai.chat.completions.create(
      model="llama3.2",
      messages=history + [{"role": "user", "content": message}],
      stream=True
  )
  reply = ""
  for chunk in stream:
    fragment = chunk.choices[0].delta.content or ""
    reply += fragment
    yield reply
  history.append({"role": "user", "content": message})

In [40]:
llama_chat('Bubbleosrt algor in python','python')

<generator object llama_chat at 0x7e9f18f387b0>

In [41]:
def mistral_chat(message, history):
  stream = openai.chat.completions.create(
      model="mistral",
      messages=history + [{"role": "user", "content": message}],
      stream=True
  )
  reply = ""
  for chunk in stream:
    fragment = chunk.choices[0].delta.content or ""
    reply += fragment
    yield reply
  history.append({"role": "user", "content": message})

In [42]:
def code_llama(message, history):
  stream = openai.chat.completions.create(
      model="codellama",
      messages=history + [{"role": "user", "content": message}],
      stream=True
  )
  reply = ""
  for chunk in stream:
    fragment = chunk.choices[0].delta.content or ""
    reply += fragment
    yield reply
  history.append({"role": "user", "content": message})

In [43]:
def llama_chat(message, history):
  stream = openai.chat.completions.create(
      model="llama3.2",
      messages=history + [{"role": "user", "content": message}],
      stream=True
  )
  reply = ""
  for chunk in stream:
    fragment = chunk.choices[0].delta.content or ""
    reply += fragment
    yield reply
  history.append({"role": "user", "content": message})

In [44]:
import threading
chat_history = []

def generate_code_response_stream(user_prompt, history, selected_model):
    """
    Generate code response using the selected model with streaming

    Args:
        user_prompt: User's input message
        history: Chat history (list of tuples)
        selected_model: Selected model name

    Yields:
        tuple: (empty string, updated history with partial response)
    """
    try:
        # Handle different models (for now, we'll use CodeQwen for all)
        if selected_model == "Code_Qwen":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
        elif selected_model == "Llama3.2":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
            result = llama_chat(user_prompt,chat_history)
            return "", chat_history
        elif selected_model == "Mistral":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
            result = mistral_chat(user_prompt,chat_history)
            return "", chat_history
        elif selected_model == "codellama":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
            result = code_llama(user_prompt,chat_history)
            return "", chat_history
        else:
            # Placeholder for other models
            system_message = f"You are a helpful code assistant using {selected_model}. "
            system_message += "Generate high-quality code and provide helpful explanations."

        # Prepare messages for the model
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ]

        # Add chat history context (last 5 exchanges)
        if history:
            recent_history = history[-5:]  # Keep last 5 exchanges for context
            for user_msg, assistant_msg in recent_history:
                messages.insert(-1, {"role": "user", "content": user_msg})
                messages.insert(-1, {"role": "assistant", "content": assistant_msg})

        # Apply chat template
        text = tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Tokenize and prepare inputs
        model_inputs = tokenizer([text], return_tensors="pt").to(device)

        # Remove token_type_ids if present (not needed for generation)
        if 'token_type_ids' in model_inputs:
            del model_inputs['token_type_ids']

        # Create TextIteratorStreamer for streaming
        streamer = TextIteratorStreamer(
            tokenizer,
            timeout=10.0,
            skip_prompt=True,
            skip_special_tokens=True
        )

        # Set up generation parameters
        generation_kwargs = dict(
            **model_inputs,
            streamer=streamer,
            max_new_tokens=1024,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )

        # Start generation in a separate thread
        thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        # Stream the response
        partial_response = ""
        for new_text in streamer:
            partial_response += new_text
            # Update history with partial response
            updated_history = history + [(user_prompt, partial_response)]
            yield "", updated_history

        # Wait for the generation thread to complete
        thread.join()

        # Final cleanup of the response
        final_response = partial_response.strip()

        # Update history with final response
        final_history = history + [(user_prompt, final_response)]
        yield "", final_history

    except Exception as e:
        error_msg = f"Error generating response: {str(e)}"
        print(error_msg)
        updated_history = history + [(user_prompt, error_msg)]
        yield "", updated_history

In [45]:
import torch
chat_history = []

def generate_code_non_response(user_prompt,history,selected_model):
    try:
        # Handle different models (for now, we'll use CodeQwen for all)
        if selected_model == "Code_Qwen":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
        elif selected_model == "Llama3.2":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
            result = llama_chat(user_prompt,chat_history)
            return "", chat_history
        elif selected_model == "Mistral":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
            result = mistral_chat(user_prompt,chat_history)
            return "", chat_history
        elif selected_model == "codellama":
            system_message = "You are a Code Generator Assistant that generates high-performance code for Windows. "
            system_message += "You can generate code, convert between programming languages, and answer coding questions. "
            system_message += "Provide clear, well-commented code with explanations when helpful."
            result = code_llama(user_prompt,chat_history)
            return "", chat_history
        else:
            # Placeholder for other models
            system_message = f"You are a helpful code assistant using {selected_model}. "
            system_message += "Generate high-quality code and provide helpful explanations."

        # Prepare messages for the model
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ]

        # Add chat history context (last 5 exchanges)
        if history:
            recent_history = history[-5:]  # Keep last 5 exchanges for context
            for user_msg, assistant_msg in recent_history:
                messages.insert(-1, {"role": "user", "content": user_msg})
                messages.insert(-1, {"role": "assistant", "content": assistant_msg})

        # Apply chat template
        text = tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Tokenize and prepare inputs
        model_inputs = tokenizer([text], return_tensors="pt").to(device)

        # Remove token_type_ids if present (not needed for generation)
        if 'token_type_ids' in model_inputs:
            del model_inputs['token_type_ids']

        # Generate response
        with torch.no_grad():
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=1024,  # Increased for longer responses
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1
            )

        # Decode the response
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Extract the assistant's response part
        if '<|im_start|>assistant' in response:
            assistant_response = response.split('<|im_start|>assistant')[-1]
            if '<|im_end|>' in assistant_response:
                assistant_response = assistant_response.split('<|im_end|>')[0]
        else:
            # Fallback: get everything after the last user message
            assistant_response = response.split(user_prompt)[-1]

        # Clean up the response
        assistant_response = assistant_response.strip()

        # Update history
        updated_history = history + [(user_prompt, assistant_response)]

        return "", updated_history

    except Exception as e:
        error_msg = f"Error generating response: {str(e)}"
        print(error_msg)
        updated_history = history + [(user_prompt, error_msg)]
        return "", updated_history






In [46]:
def clear_chat():
    """Clear the chat history"""
    return []

In [47]:
def export_chat(history):
    """Export chat history to a text file"""
    if not history:
        return "No chat history to export"

    export_text = "# CodeQwen Chat History\n\n"
    for i, (user_msg, assistant_msg) in enumerate(history, 1):
        export_text += f"## Exchange {i}\n\n"
        export_text += f"**User:** {user_msg}\n\n"
        export_text += f"**Assistant:** {assistant_msg}\n\n"
        export_text += "---\n\n"

    return export_text


In [61]:
custom_css = """
/* Import Google Fonts */
@import url('https://fonts.googleapis.com/css2?family=JetBrains+Mono:wght@400;500;700&family=Inter:wght@300;400;500;600;700&display=swap');

/* Dark theme with neon accents */
.gradio-container {
    background: linear-gradient(135deg, #0a0a0a 0%, #1a1a2e 50%, #16213e 100%);
    color: #e0e0e0;
    font-family: 'Inter', sans-serif;
}

/* Main container styling */
.contain {
    background: rgba(255, 255, 255, 0.02);
    backdrop-filter: blur(10px);
    border: 1px solid rgba(255, 255, 255, 0.1);
    border-radius: 20px;
    padding: 2rem;
    margin: 1rem;
}

/* Header styling */
.markdown h1 {
    background: linear-gradient(45deg, #00d4ff, #9146ff, #ff6b6b);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
    font-size: 2.5rem;
    font-weight: 700;
    text-align: center;
    margin-bottom: 0.5rem;
    font-family: 'Inter', sans-serif;
    text-shadow: 0 0 30px rgba(0, 212, 255, 0.3);
}

/* Chat interface styling */
.chatbot {
    background: rgba(0, 0, 0, 0.4);
    border: 1px solid rgba(0, 212, 255, 0.3);
    border-radius: 15px;
    box-shadow: 0 8px 32px rgba(0, 212, 255, 0.1);
}

.chatbot .message {
    background: rgba(255, 255, 255, 0.05);
    border-radius: 10px;
    padding: 1rem;
    margin: 0.5rem 0;
    border-left: 3px solid #00d4ff;
}

.chatbot .message.user {
    background: rgba(145, 70, 255, 0.1);
    border-left: 3px solid #9146ff;
}

/* Input field styling */
.textbox textarea {
    background: rgba(0, 0, 0, 0.6);
    border: 2px solid rgba(0, 212, 255, 0.3);
    border-radius: 12px;
    color: #e0e0e0;
    font-family: 'JetBrains Mono', monospace;
    transition: all 0.3s ease;
}

.textbox textarea:focus {
    border-color: #00d4ff;
    box-shadow: 0 0 20px rgba(0, 212, 255, 0.3);
    outline: none;
}

/* Button styling */
.btn {
    background: linear-gradient(45deg, #00d4ff, #9146ff);
    border: none;
    border-radius: 10px;
    color: white;
    font-weight: 600;
    padding: 0.8rem 1.5rem;
    transition: all 0.3s ease;
    box-shadow: 0 4px 15px rgba(0, 212, 255, 0.3);
    text-transform: uppercase;
    letter-spacing: 0.5px;
}

.btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 8px 25px rgba(0, 212, 255, 0.4);
}

.btn-secondary {
    background: linear-gradient(45deg, #ff6b6b, #ffa726);
    box-shadow: 0 4px 15px rgba(255, 107, 107, 0.3);
}

.btn-secondary:hover {
    box-shadow: 0 8px 25px rgba(255, 107, 107, 0.4);
}

/* Small button styling */
.btn-sm {
    background: rgba(0, 212, 255, 0.1);
    border: 1px solid rgba(0, 212, 255, 0.3);
    border-radius: 8px;
    color: #00d4ff;
    font-size: 0.85rem;
    padding: 0.5rem 1rem;
    margin: 0.25rem;
    transition: all 0.3s ease;
}

.btn-sm:hover {
    background: rgba(0, 212, 255, 0.2);
    transform: scale(1.05);
}

/* Dropdown styling */
.dropdown {
    background: rgba(0, 0, 0, 0.6);
    border: 2px solid rgba(145, 70, 255, 0.3);
    border-radius: 10px;
    color: #e0e0e0;
}

.dropdown:focus {
    border-color: #9146ff;
    box-shadow: 0 0 20px rgba(145, 70, 255, 0.3);
}

/* Checkbox styling */
.checkbox {
    accent-color: #00d4ff;
}

/* Sidebar styling */
.sidebar {
    background: rgba(0, 0, 0, 0.3);
    border: 1px solid rgba(255, 255, 255, 0.1);
    border-radius: 15px;
    padding: 1.5rem;
    backdrop-filter: blur(5px);
}

/* Section headers */
.markdown h3 {
    color: #00d4ff;
    font-size: 1.2rem;
    font-weight: 600;
    margin-bottom: 1rem;
    text-shadow: 0 0 10px rgba(0, 212, 255, 0.5);
}

/* Animations */
@keyframes glow {
    0%, 100% { box-shadow: 0 0 20px rgba(0, 212, 255, 0.3); }
    50% { box-shadow: 0 0 30px rgba(0, 212, 255, 0.6); }
}

.chatbot {
    animation: glow 3s ease-in-out infinite;
}

/* Scrollbar styling */
::-webkit-scrollbar {
    width: 8px;
}

::-webkit-scrollbar-track {
    background: rgba(0, 0, 0, 0.3);
    border-radius: 10px;
}

::-webkit-scrollbar-thumb {
    background: linear-gradient(45deg, #00d4ff, #9146ff);
    border-radius: 10px;
}

::-webkit-scrollbar-thumb:hover {
    background: linear-gradient(45deg, #9146ff, #ff6b6b);
}

/* Export output styling */
.export-output {
    background: rgba(0, 0, 0, 0.8);
    border: 1px solid rgba(255, 107, 107, 0.3);
    border-radius: 10px;
    font-family: 'JetBrains Mono', monospace;
    font-size: 0.9rem;
}

/* Responsive design */
@media (max-width: 768px) {
    .gradio-container {
        padding: 1rem;
    }

    .markdown h1 {
        font-size: 2rem;
    }

    .btn {
        padding: 0.6rem 1rem;
        font-size: 0.9rem;
    }
}
"""

# Create the enhanced UI
with gr.Blocks(
    title="🚀 कोड-Qwen AI Studio",
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="purple",
        neutral_hue="slate"
    ),
    css=custom_css
) as ui:

    # Header section
    gr.HTML("""
    <div style="text-align: center; margin-bottom: 2rem;">
        <h1 style="margin: 0;">🤖 कोड-Qwen AI Studio</h1>
        <p style="color: #888; font-size: 1.1rem; margin-top: 0.5rem;">
            Your AI-powered coding companion for generation, conversion & optimization
        </p>
    </div>
    """)

    with gr.Row(elem_classes="main-container"):
        # Main chat area
        with gr.Column(scale=3, elem_classes="chat-section"):
            chatbot = gr.Chatbot(
                label="💬 Chat History",
                height=500,
                show_label=True,
                container=True,
                elem_classes="chatbot",
                avatar_images=("🧑‍💻", "🤖")
            )

            with gr.Row():
                user_input = gr.Textbox(
                    label="",
                    placeholder="🚀 Ask me to generate code, explain concepts, or convert between languages...",
                    lines=2,
                    scale=5,
                    elem_classes="message-input"
                )
                send_btn = gr.Button(
                    "Send ⚡",
                    variant="primary",
                    scale=1,
                    elem_classes="send-button"
                )

        # Control panel
        with gr.Column(scale=1, elem_classes="sidebar"):
            gr.HTML("<div style='text-align: center; margin-bottom: 1rem;'>⚙️ <b>Control Panel</b></div>")

            # Model selection
            model_dropdown = gr.Dropdown(
                choices=["🔥 Code_Qwen", "🦙 Llama3.2", "🌟 Mistral", "💫 CodeLlama", "⚡ Phi3"],
                label="🎯 Select Model",
                value="🔥 Code_Qwen",
                info="Choose your AI coding assistant",
                elem_classes="model-dropdown"
            )

            # Settings
            with gr.Group():
                gr.HTML("<div style='text-align: center; margin: 1rem 0;'>📊 <b>Settings</b></div>")

                streaming_toggle = gr.Checkbox(
                    label="🌊 Enable Streaming",
                    value=True,
                    info="Stream responses in real-time"
                )

                temperature_slider = gr.Slider(
                    minimum=0.1,
                    maximum=1.0,
                    value=0.7,
                    step=0.1,
                    label="🌡️ Temperature",
                    info="Controls creativity"
                )

            # Quick examples
            with gr.Group():
                gr.HTML("<div style='text-align: center; margin: 1rem 0;'>🎯 <b>Quick Examples</b></div>")

                example_buttons = [
                    gr.Button("🐍 Python Sorting Algorithm", size="sm", elem_classes="example-btn"),
                    gr.Button("🔄 Python ➜ JavaScript", size="sm", elem_classes="example-btn"),
                    gr.Button("🔍 Explain Recursion", size="sm", elem_classes="example-btn"),
                    gr.Button("🌐 Flask REST API", size="sm", elem_classes="example-btn"),
                    gr.Button("⚡ Optimize Performance", size="sm", elem_classes="example-btn"),
                    gr.Button("🛠️ Debug Code", size="sm", elem_classes="example-btn")
                ]

            # Chat controls
            with gr.Group():
                gr.HTML("<div style='text-align: center; margin: 1rem 0;'>🛠️ <b>Chat Controls</b></div>")

                with gr.Row():
                    clear_btn = gr.Button("🗑️ Clear", variant="secondary", scale=1)
                    export_btn = gr.Button("📤 Export", variant="secondary", scale=1)

                # Stats display
                gr.HTML("""
                <div style='text-align: center; margin-top: 1rem; padding: 1rem; background: rgba(0,0,0,0.3); border-radius: 10px;'>
                    <div style='color: #00d4ff; font-size: 0.9rem;'>📈 <b>Session Stats</b></div>
                    <div style='color: #888; font-size: 0.8rem; margin-top: 0.5rem;'>
                        Messages: <span id='msg-count'>0</span> |
                        Tokens: <span id='token-count'>0</span>
                    </div>
                </div>
                """)

    # Export output (hidden by default)
    export_output = gr.Textbox(
        label="📁 Exported Chat History",
        lines=12,
        visible=False,
        elem_classes="export-output"
    )

    # Footer
    gr.HTML("""
    <div style="text-align: center; margin-top: 2rem; padding: 1rem; border-top: 1px solid rgba(255,255,255,0.1);">
        <p style="color: #666; font-size: 0.9rem;">
            🚀 Built with CodeQwen • Powered by Gradio • Made with ❤️
        </p>
    </div>
    """)


    # Event handlers
    def send_message(message, history, model, streaming_enabled):
        if message.strip():
            if streaming_enabled:
                # Use streaming generator
                yield from generate_code_response_stream(message.strip(), history, model)
            else:
                # Use non-streaming version
                result = generate_code_non_response(message.strip(), history, model) # Corrected function call
                yield "", result # Yield as tuple to match streaming function output structure
        else:
            yield "", history

    def set_example(example_text):
        return example_text

    # Wire up the events with streaming support
    send_btn.click(
        fn=send_message,
        inputs=[user_input, chatbot, model_dropdown, streaming_toggle],
        outputs=[user_input, chatbot]
    )

    user_input.submit(
        fn=send_message,
        inputs=[user_input, chatbot, model_dropdown, streaming_toggle],
        outputs=[user_input, chatbot]
    )

    clear_btn.click(
        fn=clear_chat,
        outputs=[chatbot]
    )

    export_btn.click(
        fn=export_chat,
        inputs=[chatbot],
        outputs=[export_output]
    ).then(
        fn=lambda: gr.update(visible=True),
        outputs=[export_output]
    )

    # Example button handlers
    example_texts = [
        "Generate a Python sorting algorithm with time complexity analysis",
        "Convert this Python function to JavaScript: def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
        "Explain recursion with practical examples and show iterative alternatives",
        "Create a REST API in Flask with CRUD operations for a todo list",
        "Suggest ways to optimize the performance of a Python loop", # Added text
        "Help debug this C++ code snippet that's causing a segmentation fault" # Added text
    ]
    for i, btn in enumerate(example_buttons):
        btn.click(
            fn=set_example,
            inputs=[gr.State(example_texts[i])],
            outputs=[user_input]
        )

/tmp/ipython-input-61-3142111160.py:233: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


In [62]:
if __name__== "__main__":
    ui.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f604c1b6c97c65ede.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
